In [ ]:
!pip install -q transformers

This command installs the Hugging Face `transformers` library using a "quiet" mode, which reduces the amount of text output you see during the installation.


* `!`: The exclamation mark at the beginning tells a development environment, like a Jupyter notebook or Google Colab, to run the line as a **shell command** rather than as a line of Python code.
* `pip install`: This is the core command for **pip**, the standard package installer for Python. It tells the system to find, download, and install a specific software package.
* `-q`: This flag stands for **quiet**. It prevents the terminal from printing the large amount of informational messages that normally appear during an installation, showing only essential output like errors or warnings.
* `transformers`: This is the name of the Python library being installed. The **Hugging Face `transformers` library** is an essential toolkit for machine learning, providing access to thousands of pre-trained models for tasks involving text, images, and audio, especially in Natural Language Processing (NLP).

In [ ]:
from transformers import AutoProcessor, ShieldGemma2ForImageClassification
from PIL import Image
import requests
import torch
import gradio as gr
import os

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"


This script begins by importing several key Python libraries required for the task:

* `from transformers import AutoProcessor, ShieldGemma2ForImageClassification`: This imports two components from the **Hugging Face `transformers` library**.
    * `ShieldGemma2ForImageClassification` is the specific **machine learning model** being used, which is a version of Google's Gemma-2 model designed for classifying images.
    * `AutoProcessor` is a convenient tool that automatically loads the correct data pre-processor for the model, ensuring images are in the right format and size before being fed into the model.

* `from PIL import Image`: This imports the `Image` module from the **Pillow library** (`PIL`). Pillow is the standard library in Python for opening, manipulating, and saving images.

* `import requests`: This imports a popular library for making **HTTP requests**, which is essential for downloading data, such as images, from a URL.

* `import torch`: This imports **PyTorch**, a leading open-source machine learning framework that the `transformers` library is built upon. It provides the core functionalities for tensor computations and running models on GPUs.

* `import gradio as gr`: This imports the **Gradio library** with the alias `gr`. Gradio is used to quickly build and share simple web UIs for machine learning models, making them easy to test and demonstrate.

* `import os`: This imports Python's built-in **operating system** module, which allows the code to interact with the OS, including setting environment variables.

Finally, the line `os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"` sets an environment variable to activate **`hf-transfer`**. This is a high-performance, Rust-based utility that significantly accelerates the download speed of models and datasets from the Hugging Face Hub, which is especially useful for very large models.

In [ ]:
class CFG:
    model = "google/shieldgemma-2-4b-it"
    device = 'cuda'

This code defines a configuration class named `CFG` to store important settings for a machine learning script in a single, organized place.

* `model = "google/shieldgemma-2-4b-it"`: This line sets the **model identifier**. The string `"google/shieldgemma-2-4b-it"` tells the `transformers` library to download and use a specific model from the Hugging Face Hub. In this case, it's the **ShieldGemma-2** model from Google, which has 4 billion parameters and is instruction-tuned.

* `device = 'cuda'`: This line specifies the **hardware device** on which the model will run. Setting the device to `'cuda'` tells the script to use a **NVIDIA GPU** for all calculations. Running a model on a GPU is significantly faster than using a CPU. GPU. 🚀

In [ ]:
model = ShieldGemma2ForImageClassification.from_pretrained(CFG.model, device_map = CFG.device)
processor = AutoProcessor.from_pretrained(CFG.model)

This code loads the pre-trained ShieldGemma-2 model and its associated data processor from the Hugging Face Hub.


* `model = ShieldGemma2ForImageClassification.from_pretrained(CFG.model, device_map=CFG.device)`: This line initializes the **machine learning model**.
    * The `from_pretrained()` method is a key feature of the `transformers` library. It automatically downloads the model weights and configuration from the Hugging Face Hub using the identifier stored in `CFG.model` (`"google/shieldgemma-2-4b-it"`).
    * The `device_map=CFG.device` argument tells the library to load the model directly onto the specified hardware, which in this case is the NVIDIA **GPU** (`'cuda'`). This ensures all computations will be performed on the faster hardware.
* `processor = AutoProcessor.from_pretrained(CFG.model)`: This line loads the **data processor** that corresponds to the model.
    * For an image model, the processor is responsible for converting an input image into the numerical format (a tensor) that the model can understand. It handles tasks like resizing, normalizing pixel values, and structuring the data correctly.
    * Using `AutoProcessor.from_pretrained()` with the same model identifier guarantees that the pre-processing steps are perfectly matched to the model's requirements.

In [ ]:
def infer(image, policies, policy_descriptions):
  policies = policies.split(";")
  policy_descriptions = policy_descriptions.split(";")
  custom_policies = dict(zip(policies, policy_descriptions))

  inputs = processor(
      images=[image], custom_policies=custom_policies,
      policies=policies, return_tensors="pt",
  ).to(model.device)

  with torch.inference_mode():
      output = model(**inputs)


  outs = {}

  for idx, policy in enumerate(output.probabilities.cpu()):
    yes_prob = policy[0]
    outs[f"Yes for {policies[idx]}"] = yes_prob

  return outs

This Python function, named `infer`, is designed to perform inference using the `ShieldGemma-2` model. It takes an image and a set of custom rules (policies) and calculates the probability that the image complies with each rule.

1.  **Parse Inputs**:
    The function first takes two strings, `policies` and `policy_descriptions`, which are expected to contain items separated by semicolons. It splits these strings into lists and then uses `zip` to combine them into a single Python **dictionary** called `custom_policies`. This dictionary maps each policy name to its description.

2.  **Pre-process Data**:
    Next, it uses the `processor` object (loaded in a previous step) to prepare the inputs for the model. The processor takes the input `image`, the `custom_policies` dictionary, and the list of `policies`. It converts the image into the correct numerical format (a tensor) and bundles it with the policy information. The `return_tensors="pt"` argument ensures the output is a PyTorch tensor, and `.to(model.device)` moves this data to the **GPU** (`'cuda'`) to match the model's location.

3.  **Run Inference**:
    This step runs the actual prediction.
    * `with torch.inference_mode():` is a performance optimization. It tells PyTorch that no training is occurring, which makes the computation faster and more memory-efficient.
    * `output = model(**inputs)` feeds the processed inputs into the `model`. The `**` syntax unpacks the `inputs` dictionary into the arguments the model expects. The model then evaluates the image against each policy.

4.  **Format the Output**:
    Finally, the function creates a clean, human-readable output. It initializes an empty dictionary `outs`. It then loops through the `output.probabilities` from the model, moving them back to the CPU with `.cpu()`. For each policy, it extracts the probability for the "Yes" case (e.g., the probability that the image *does* comply with the policy) and adds it to the `outs` dictionary with a descriptive key like `"Yes for [policy_name]"`.

The function returns the `outs` dictionary, which contains the final probability scores for each policy.

In [ ]:
with gr.Blocks() as demo:
  with gr.Row():
    with gr.Column():
      image = gr.Image(type="pil")
      policies = gr.Textbox(label = "Names of policies - separated by semicolon")
      policy_descriptions = gr.Textbox(label = "Descriptions of policies - separated by semicolon", lines=10)
      btn = gr.Button("Submit")
    with gr.Column():
      outputs = gr.Label()

    btn.click(fn=infer, inputs=[image, policies, policy_descriptions], outputs=outputs)

  gr.Examples(examples = [  ["Sex ; Violence ",
                           "The image shall not contain content that depicts explicit or graphic sexual acts.; The image shall not contain content that depicts shocking, sensational, or gratuitous violence   (e.g., excessive blood and gore, gratuitous violence against animals, extreme injury or moment of death)."],


                              ["Sex ; Violence ",
                           """The image shall not contain content that depicts explicit or graphic sexual acts (e.g., pornography, erotic content, sexual intercourse, genital exposure, or depictions of rape or sexual assault).  Classical art, educational imagery, or non-sexualized nudity—such as depictions found in the Sistine Chapel or anatomy textbooks—are not considered sexually explicit for moderation purposes;  The image shall not contain content that depicts or glorifies graphic violence or physical harm (e.g., torture, murder, dismemberment, or depictions of extreme bodily injury). Images related to historical events, war documentation, or journalistic reporting that are not gratuitous or sensationalized are generally acceptable unless they focus on explicit gore or suffering."""],

                          ],
              inputs = [policies, policy_descriptions])

This code uses the **Gradio library** (`gr`) to build a web user interface (UI) for the image classification model. It creates an interactive demo where a user can upload an image, define safety policies, and see the model's evaluation.

***

## UI Layout and Components

The interface is constructed using `gr.Blocks`, which allows for a custom layout of components.

* `with gr.Row():` creates the main horizontal layout, which is split into two columns.
* **Left Column (Inputs)**: This column contains all the user input fields.
    * `gr.Image(type="pil")`: An **image upload box**. `type="pil"` ensures the uploaded file is converted into a format the model's processor can handle.
    * `gr.Textbox()`: Two **text boxes** are created. The first is for short policy names (e.g., "Violence"), and the second, larger box (`lines=10`) is for their detailed descriptions.
    * `gr.Button("Submit")`: A clickable **button** to start the analysis.
* **Right Column (Outputs)**: This column is for displaying the results.
    * `gr.Label()`: A **label component** that is designed to neatly display classification results, which will show the "Yes" probability for each policy.

***

## Interactivity

The line `btn.click(fn=infer, inputs=[image, policies, policy_descriptions], outputs=outputs)` is the core of the UI's functionality. It connects the components:

* When the `Submit` button (`btn`) is clicked...
* ...it calls the `infer` function...
* ...using the content from the `image`, `policies`, and `policy_descriptions` components as input...
* ...and sends the function's returned dictionary to the `outputs` label for display.

***

## Pre-filled Examples

Finally, `gr.Examples(...)` adds a section at the bottom of the UI. This provides users with pre-written examples for the policy and description fields, making it easy to test the application without having to type out long safety policies manually. Users can simply click an example to populate the fields and then upload an image to test.

In [ ]:
demo.launch(debug=True)

This command starts the web server for the Gradio application, making the user interface accessible in a web browser.

***

## Method Breakdown

* `demo.launch()`: This is the standard method used to run any Gradio interface. When called, it starts a local web server and provides a URL (usually `http://127.0.0.1:7860`) that you can open in your browser to interact with the UI you've built.

* `debug=True`: This argument enables **Gradio's debug mode**. When `debug=True`, any errors that occur while the application is running will be printed directly in your browser. This is incredibly useful for developers to find and fix issues with the code, such as problems within the `infer` function. 🚀